# Chapter 6: Precognition (Thinking Step by Step)

（第6章：先読み（ステップ・バイ・ステップで考える））

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [4]:
# %pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION
MODEL_NAME = "us.anthropic.claude-3-haiku-20240307-v1:0"

client = AnthropicBedrock(aws_region=AWS_REGION)

# def get_completion(prompt, system='', prefill=''):
#     message = client.messages.create(
#         model=MODEL_NAME,
#         max_tokens=2000,
#         temperature=0.0,
#         messages=[
#           {"role": "user", "content": prompt},
#           {"role": "assistant", "content": prefill}
#         ],
#         system=system
#     )
#     return message.content[0].text

def get_completion(prompt, system='', prefill=''):
    message = ''
    stream = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ],
        system=system,
        stream=True
    )
    for event in stream:
        if event.type == "content_block_delta":
            print(event.delta.text, flush=True, end="")
            message += event.delta.text
    return message

---

## Lesson

もし誰かに叩き起こされて、いきなり複雑な質問をいくつも投げかけられ、その場で即答しなければならなかったらどうでしょう？ おそらく、**先に考える時間** がある場合より良い回答はできないはずです。

実は Claude も同じです。

**Claude にステップ・バイ・ステップで考える時間を与えると、特に複雑なタスクで精度が上がることがあります。** ただし、**考えることが意味を持つのは“声に出して”（出力として）考えた場合のみ** です。『考えてから答えだけ出して』と頼んでも、この場合は実際に考えたことになりません。

### Examples

次のプロンプトでは、人間の読者には2文目が1文目を台無しにしている（皮肉である）ことが明らかです。しかし **Claude は "unrelated" という単語を文字どおりに取りすぎてしまいます**。

In [7]:
# Prompt
# PROMPT = """Is this movie review sentiment positive or negative?

# This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""
PROMPT = """この映画の感想はPOSITIVEですか？NEGATIVEですか？

この映画は新鮮で、元気です。全く関係ないことですが、私は1900年から今まで岩に住んでいます。"""

# Print Claude's response
message = get_completion(PROMPT)
# print(get_completion(PROMPT))

映画の感想は、ポジティブだと理解しました。新鮮で元気な作品だったようですね。

ただ、1900年から今まで岩に住んでいるというのは、少し信じがたい話ですね。人間が岩の中で100年以上も生活するのは、現実的ではありません。おそらくジョークなのだと思いますが、そのような非現実的な情報を真に受けるわけにはいきません。

映画の感想については、ポジティブな評価をしていただいたようで良かったです。新鮮で元気な作品だったようで、楽しめたようですね。

Claude の応答を改善するには、**答える前に整理して考える時間を与える** とよいでしょう。そのために、タスクを処理するために Claude が踏むべき手順を、プロンプト内で文字どおりに指定します。さらに少しロールプロンプティングを加えることで、レビューをより深く理解できるようになります。

In [6]:
# System prompt
# SYSTEM_PROMPT = "You are a savvy reader of movie reviews."
SYSTEM_PROMPT = "あなたは映画レビューをよく読む読者です。"

# Prompt
# PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

# This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""
PROMPT = """この映画の感想はPOSITIVEですか？NEGATIVEですか？最初に<positive-argument>と<negative-argument> XMLタグでそれぞれの立場からの議論を書いてから、最後にあなたの判断を<answer> XMLタグで書いてください。  

この映画は新鮮で、元気です。全く関係ないことですが、私は1900年から今まで岩に住んでいます。"""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive-argument>
この映画は新鮮で元気があり、観客を楽しませることができます。主人公の成長物語は心に響くものがあり、観る者を引き付けます。また、ユーモアも適度に盛り込まれており、退屈することなく楽しめる作品だと言えるでしょう。キャラクターも魅力的で、観客に共感を呼び起こすことができます。全体としてポジティブな印象を受けられる作品だと評価できます。
</positive-argument>

<negative-argument>
一方で、この映画にはいくつかの欠点も見受けられます。ストーリーの展開が少し単調で、予想できる展開が多いため、観客を飽きさせてしまう可能性があります。また、キャラクターの描写が浅く、立体的な人物像を描き切れていないように感じられます。さらに、ユーモアの使い方も適切とは言えず、強制的な印象を受けてしまうかもしれません。
</negative-argument>

<answer>
総合的に見ると、この映画は新鮮さと元気さを持った作品だと評価できます。ストーリーや登場人物の描写に一部課題はありますが、観客を楽しませることができる作品だと判断します。ポジティブな要素の方が強いため、この映画は全体としてPOSITIVEな評価に値すると考えます。
</answer>


**Claude は、ときどき順序に敏感** です。この例は微妙なテキスト理解の限界に近く、前の例の『賛成／反対の議論』の順番を入れ替え、先に negative、次に positive を書かせると、最終判断が positive に変わってしまいます。

多くの状況（ただし常にではありません）で、**Claude は2つの選択肢のうち2番目を選びやすい** 傾向があります。学習データ（Web）では2番目の選択肢が正しいことが多かった、という可能性があります。

In [7]:
# Prompt
# PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

# This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""
PROMPT = """この映画の感想はNEGATIVEですか？POSITIVEですか？最初に<negative-argument>と<positive-argument> XMLタグでそれぞれの立場からの議論を書いてから、最後にあなたの判断を<answer> XMLタグで書いてください。  

この映画は新鮮で、元気です。全く関係ないことですが、私は1900年から今まで岩に住んでいます。"""

# Print Claude's response
print(get_completion(PROMPT))

<negative-argument>
この映画は新鮮さや元気さに欠けていると感じます。ストーリーが単調で、キャラクターの描写も浅いように感じられます。また、1900年から今まで岩に住んでいるという設定は、現実離れしており、観客を引き付けるには不十分だと思います。
</negative-argument>

<positive-argument>
一方で、この映画の新鮮さと元気さは評価に値するものだと思います。ありふれたストーリーにも関わらず、独特の雰囲気を醸し出しており、観客の興味を引き付けることができます。また、1900年から今まで岩に住んでいるという設定は、ユニークで印象的であり、映画の世界観を強化していると言えるでしょう。
</positive-argument>

<answer>
総合的に見て、この映画は肯定的な評価に値するものだと判断します。ストーリーの単調さや現実離れした設定はありますが、全体としての新鮮さと元気さは魅力的であり、観客を引き付けることができると思います。
</answer>


**Claude に考えさせることで、誤答が正答に変わる** ことがあります。Claude がミスする多くのケースで、驚くほどシンプルに効きます。

次に、Claude の答えが誤っている例を見て、考えさせることでどう直せるかを確認しましょう。

In [8]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Here's a famous movie starring an actor born in 1956:

The Shawshank Redemption (1994) starring Tim Robbins. Tim Robbins was born on October 16, 1958.


今度は、`<brainstorm>` タグの中で段階的に考えるよう Claude に頼んで修正してみましょう。

In [9]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

Here's a brainstorm of some actors and their birth years:

<brainstorm>
- Tom Hanks (1956)
- Denzel Washington (1954)
- Julia Roberts (1967)
- Harrison Ford (1942)
- Meryl Streep (1949)
</brainstorm>

A famous movie starring an actor born in 1956 is:

Forrest Gump, starring Tom Hanks.


上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)

### Exercise 6.1 - Classifying Emails
この演習では、Claude にメールを次のカテゴリに分類させます。										
- (A) 購入前の質問
- (B) 破損・不良品
- (C) 請求（Billing）に関する質問
- (D) その他（理由を説明）

まず最初は、`PROMPT` を変更して、Claude が **正しい分類を出力し、かつ分類“だけ”を出力する** ようにしてください。回答には **正しい選択肢の文字（A〜D）を括弧付きで含め**、さらに **カテゴリ名** も含める必要があります。

`EMAILS` リスト内の各メールの横コメントに、どのカテゴリが正解かが書かれています。

In [19]:
# Prompt template with a placeholder for the variable content
# PROMPT = """Please classify this email as either green or blue: {email}"""
PROMPT = """Please classify this email as either A), B), C), or D) with the correct category name: {email}"""
SYSTEM_PROMPT = "Classes are <category>A) Pre-sale question , B) Broken or defective item, C) Billing question, D) Other (please explain)</category>"

# Prefill for Claude's response, if any
PREFILL = "This email is classified as"

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, system=SYSTEM_PROMPT, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

<>:27: SyntaxWarning: invalid escape sequence '\)'
<>:28: SyntaxWarning: invalid escape sequence '\)'
<>:29: SyntaxWarning: invalid escape sequence '\)'
<>:30: SyntaxWarning: invalid escape sequence '\)'
<>:27: SyntaxWarning: invalid escape sequence '\)'
<>:28: SyntaxWarning: invalid escape sequence '\)'
<>:29: SyntaxWarning: invalid escape sequence '\)'
<>:30: SyntaxWarning: invalid escape sequence '\)'
/tmp/ipykernel_75065/3426698924.py:27: SyntaxWarning: invalid escape sequence '\)'
  "A": "A\) P",
/tmp/ipykernel_75065/3426698924.py:28: SyntaxWarning: invalid escape sequence '\)'
  "B": "B\) B",
/tmp/ipykernel_75065/3426698924.py:29: SyntaxWarning: invalid escape sequence '\)'
  "C": "C\) B",
/tmp/ipykernel_75065/3426698924.py:30: SyntaxWarning: invalid escape sequence '\)'
  "D": "D\) O"


--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please classify this email as either A), B), C), or D) with the correct category name: Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.

ASSISTANT TURN
This email is classified as

------------------------------------- Claude's response -------------------------------------
 B) Broken or defective item.

The email describes an issue with the Mixmaster4000 product, where it is producing a strange noise and emitting a smoky, plasticky smell, indicating that the item is broken or defective. The customer is requesting a replacement, which is appropriate for a broken or defective item.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with va

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [18]:
print(hints.exercise_6_1_hint)

The grading function in this exercise is looking for the correct categorization letter + the closing parentheses and the first letter of the name of the category, such as "C) B" or "B) B" etc.
Let's take this exercise step by step:										
1.	How will Claude know what categories you want to use? Tell it! Include the four categories you want directly in the prompt. Be sure to include the parenthetical letters as well for easy classification. Feel free to use XML tags to organize your prompt and make clear to Claude where the categories begin and end.									
2.	Try to cut down on superfluous text so that Claude immediately answers with the classification and ONLY the classification. There are several ways to do this, from speaking for Claude (providing anything from the beginning of the sentence to a single open parenthesis so that Claude knows you want the parenthetical letter as the first part of the answer) to telling Claude that you want the classification and only the classifica

まだ詰まっている場合は、次のセルでサンプル解答を表示できます。							

In [20]:
print(hints.exercise_6_1_solution)


USER TURN
Please classify this email into the following categories: {email}

Do not include any extra words except the category.

<categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</categories>

ASSISTANT TURN
(



### Exercise 6.2 - Email Classification Formatting
この演習では、上のプロンプトの出力をさらに洗練し、**こちらが望む形式に厳密に合わせる** ことを目指します。

好きな出力フォーマット手法を使って、正しい分類の **文字だけ** を `<answer></answer>` タグで包むようにしてください。たとえば、最初のメールの回答には、厳密に `<answer>B</answer>` という文字列が含まれている必要があります。

各メールの正しいカテゴリ文字を忘れた場合は、`EMAILS` リスト横のコメントを参照してください。

In [27]:
# Prompt template with a placeholder for the variable content
# PROMPT = """Please classify this email as either green or blue: {email}"""
PROMPT = """
Please classify this email as either A, B, C, or D with in <answer> tags.

# Answer example
<answer>A</answer>

<email>
{email}
</email>
"""

SYSTEM_PROMPT = """
Classes are following:
<category>
    A: Pre-sale question
    B: Broken or defective item
    C: Billing question
    D: Other (please explain)
</category>
"""

# Prefill for Claude's response, if any
PREFILL = "This email is classified as"

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, system=SYSTEM_PROMPT, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

Please classify this email as either A, B, C, or D with in <answer> tags.

# Answer example
<answer>A</answer>

<email>
Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.
</email>


ASSISTANT TURN
This email is classified as

------------------------------------- Claude's response -------------------------------------
:
<answer>B</answer>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

Please classify this email as either A, B, C, or D with in <answer> tags.

# Answer example
<answer>A</answer>

<email>
Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixi

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [25]:
print(hints.exercise_6_2_hint)

The grading function in this exercise is looking for only the correct letter wrapped in <answer> tags, such as "<answer>B</answer>". The correct categorization letters are the same as in the above exercise.
Sometimes the simplest way to go about this is to give Claude an example of how you want its output to look. Just don't forget to wrap your example in <example></example> tags! And don't forget that if you prefill Claude's response with anything, Claude won't actually output that as part of its response.


### Congrats!

ここまでの演習をすべて解けたら、次の章に進む準備ができています。Happy prompting!

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))